In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import gc
import time
import csv
import matplotlib.pyplot as plt

In [2]:
use_gpu = torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu")
SEED = 1234
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)

In [3]:
class WeatherTrainingDataset(Dataset):
    def __init__(self, X, y, augment = None):
        self.X = X
        self.y = y
        self.augment = augment
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.augment is not None:
            img = self.augment(self.X[idx])
        else:
            img = self.X[idx]
        label = self.y[idx]
        return img, label

class WeatherTestingDataset(Dataset):
    def __init__(self, X, augment = None):
        self.X = X
        self.augment = augment
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.augment is not None:
            img = self.augment(self.X[idx])
        else:
            img = self.X[idx]
        return img

In [4]:
class WeatherRegrssionNet(nn.Module):
    def __init__(self):
        super(WeatherRegrssionNet, self).__init__()

        ###########################general information###########################
        self.conv1_out_channels = 64
        self.leakyReLU1_slope = 0.05
        self.conv1_dropout_rate = 0.2

        self.conv0 = nn.Sequential(
            nn.Conv2d(13, self.conv1_out_channels, kernel_size=5, padding=2),
            nn.LeakyReLU(negative_slope=self.leakyReLU1_slope),
            nn.BatchNorm2d(self.conv1_out_channels),
            nn.Dropout2d(0.2)
        )
        self.conv1 = nn.Sequential(
            nn.Conv2d(self.conv1_out_channels, self.conv1_out_channels, kernel_size=5, padding=2),
            nn.LeakyReLU(negative_slope=self.leakyReLU1_slope),
            nn.BatchNorm2d(self.conv1_out_channels),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.25)
        )

        #########################location information######################
        self.conv2_out_channels = 512
        self.leakyReLU2_slope = 0.05
        self.conv2_dropout_rate = 0.5
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(self.conv1_out_channels, 128, kernel_size=3, padding=1),
            nn.LeakyReLU(negative_slope=self.leakyReLU2_slope),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.3)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, self.conv2_out_channels, kernel_size=3, padding=1),
            nn.LeakyReLU(negative_slope=self.leakyReLU2_slope),
            nn.BatchNorm2d(self.conv2_out_channels),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.4)
        )
        #######################Fully connected layer########################
        self.fc1 = nn.Sequential(
            nn.Linear(3*4*self.conv2_out_channels, self.conv2_out_channels),
            nn.ReLU(),
            nn.BatchNorm1d(self.conv2_out_channels),
            nn.Dropout(0.4)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(self.conv2_out_channels, self.conv2_out_channels),
            nn.ReLU(),
            nn.BatchNorm1d(self.conv2_out_channels),
            nn.Dropout(0.4)
        )
        self.fc3 = nn.Linear(self.conv2_out_channels, 2)

    def forward(self, x):
        x = self.conv0(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(-1, 3*4*self.conv2_out_channels)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [5]:
def init_weights(m):
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

def train(train_loader, model, loss_fn, optimizer, device):
    model.train()
    gc.collect()
    torch.cuda.empty_cache()
    train_loss = []
    train_acc = []
    for (img, label) in train_loader:
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = model(img)
        loss = loss_fn(output, label)
        loss.backward()            
        optimizer.step()
        train_loss.append(loss.item())
        #release memory
        del img, label, output, loss
        torch.cuda.empty_cache()
    train_loss = np.mean(train_loss)
    return train_loss
    
    
def valid(valid_loader, model, loss_fn, device):
    model.eval()
    with torch.no_grad():
        valid_loss = []
        valid_acc = []
        for (img, label) in valid_loader:
            img = img.to(device)
            label = label.to(device)
            output = model(img)
            loss = loss_fn(output, label)
            valid_loss.append(loss.item())
            #release memory
            del img, label, output, loss
            torch.cuda.empty_cache()
    #average over all batches
    valid_loss = np.mean(valid_loss)
    gc.collect()
    torch.cuda.empty_cache()
    return valid_loss

In [6]:
def predict(data_loader, model, device, isValid = False, isTesting = False):
    model.eval()
    with torch.no_grad():
        predict_result = []
        if isValid:
            for (img, label) in data_loader:
                img = img.to(device)
                output = model(img).cpu().detach().numpy()
                predict_result.extend(output)
        if isTesting:
            for img in data_loader:
                img = img.to(device)
                output = model(img).cpu().detach().numpy()
                predict_result.extend(output)
    return predict_result

def write_to_csv(predict_result, file_name):
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['INDEX', 'PM2.5-1', 'PM2.5-2'])
        for i in range(len(predict_result)):
            writer.writerow([str(i+1), str(predict_result[i][0]), str(predict_result[i][1])])

In [7]:
class WeatherTrainingRNNDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        return X, y

class WeatherTestingRNNDataset(Dataset):
    def __init__(self, X):
        self.X = X
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = self.X[idx]
        return X

In [8]:
class WeatherLstmNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(WeatherLstmNet, self).__init__()
        self.num_layers = num_layers 
        self.input_size = input_size 
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, dropout = 0.3)
        self.fc1 = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_size),
            nn.Dropout(0.3)
        )
        self.fc2 = nn.Linear(hidden_size, 2)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(device).double()
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(device).double()
            
        # Propagate input through LSTM
        out, (h_out, c_out) = self.lstm(x, (h_0, c_0)) 
        out = out[:, -1, :] #(batch_size, hidden_size x D)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

# Steps
- Load CNN model and predict result
- data preprocessing
- train RNN

In [9]:
X_train = np.load('../data/training_x_grid.npy')
X_test = np.load('../data/testing_x_grid.npy')
y_train = np.load('../data/training_y.npy')

In [10]:
mean_img = np.mean(X_train, axis=(0, 1, 2))
std_img = np.std(X_train, axis=(0, 1, 2))

## Predict by CNN

In [11]:
CNN_model_path = "b06702064_ShengYenLin_CNN.pth"
checkpoint = torch.load(CNN_model_path)
CNN_model = WeatherRegrssionNet()
CNN_model.load_state_dict(checkpoint['model_state_dict'])
CNN_model.to(device)
CNN_model = CNN_model.double()

In [12]:
image_transforms = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=mean_img, std=std_img)
                ])

batch_size = 128

#predict training set
train_dataset = WeatherTrainingDataset(X_train, y_train, image_transforms)
data_loader_CNN = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)  
y_pred_train = predict(data_loader_CNN, CNN_model, device, isValid = True)
y_pred_train = np.array(y_pred_train)

#predict testing set
test_dataset_CNN = WeatherTestingDataset(X_test, image_transforms)
test_loader_CNN = DataLoader(test_dataset_CNN, batch_size=batch_size, shuffle=False) 
y_pred_test = predict(test_loader_CNN, CNN_model, device, isTesting = True)

In [13]:
#normalize RNN dataset
SS = StandardScaler()
y_pred_train_SS = SS.fit_transform(y_pred_train)
y_pred_test_SS = SS.transform(y_pred_test)

## Prepare RNN dataset

In [14]:
time_step = 8
X_train_RNN, X_test_RNN = ([] for _ in range(2))

for i in range(y_pred_train_SS.shape[0]-time_step):
    X_train_RNN.append(y_pred_train_SS[i:i+time_step])
    X_test_RNN.append(y_pred_test_SS[i:i+time_step])
X_train_RNN = np.array(X_train_RNN)
X_test_RNN  = np.array(X_test_RNN)
y_train_RNN = y_train[time_step:] 

In [15]:
#Data set
test_ratio = 0.25
X_train_RNN, X_valid_RNN, y_train_RNN, y_valid_RNN = train_test_split(X_train_RNN, y_train_RNN, test_size = test_ratio)
train_dataset_RNN = WeatherTrainingRNNDataset(X_train_RNN, y_train_RNN)
valid_dataset_RNN = WeatherTrainingRNNDataset(X_valid_RNN, y_valid_RNN)
test_dataset_RNN = WeatherTestingRNNDataset(X_test_RNN)
#Data loader
batch_size = 128
train_loader_RNN = DataLoader(train_dataset_RNN, batch_size=batch_size, shuffle=True)
valid_loader_RNN = DataLoader(valid_dataset_RNN, batch_size=batch_size, shuffle=False)  
test_loader_RNN = DataLoader(test_dataset_RNN, batch_size=batch_size, shuffle=False)  

## Predict by RNN

In [16]:
input_size = 2 #number of features
hidden_size = 512 #number of weight in hidden layer
num_layers = 2 #number of stacked LSTM layer

RNN_model_path = "b06702064_ShengYenLin_RNN.pth"
checkpoint = torch.load(RNN_model_path)
RNN_model = WeatherLstmNet(input_size, hidden_size, num_layers)
RNN_model.load_state_dict(checkpoint['model_state_dict'])
RNN_model.to(device)
RNN_model = RNN_model.double()

In [17]:
y_pred_RNN = predict(test_loader_RNN, RNN_model, device, isTesting = True)
write_to_csv(y_pred_RNN, 'b06702064_ShengYenLin_final_test.csv')